# 1 Initialisation

## 1.1 Imports

In [ ]:
# Librairies pour le traitement des données
import pandas as pd
# Dask API is a parallelized version of Numpy, Pandas, Sklearn.
import dask.dataframe as dd #  Pour la première fois installer avec la commande 'pip install "dask[complete]"'
import numpy as np

# Librairies pour la visualisation de graphiques
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() #Définir le style par défaut pour les graphiques


# Librairies pour la visualisation de cartes
import folium # Pour la première fois installer avec la commande 'pip install folium' ou 'conda install folium -c conda-forge'. Voir : https://python-visualization.github.io/folium/quickstart.html
import branca.colormap as cm
from branca.element import Template, MacroElement

low_memory=False # faible mémoire vive (RAM)

## 1.2 Définition de fonctions utilitaires

In [ ]:
def verifier_taille(data, dask=True):
    """ Fonction de vérification de la taille d'un data set """
    lignes = data.shape[0]
    if dask:
        lignes = lignes.compute()
    colonnes = data.shape[1]
    print('Le data set contient :')
    print('\t-', lignes, 'lignes et', colonnes, 'colonnes.')

    nb_null = data.isnull().sum().sum()
    if dask:
        nb_null = nb_null.compute()
    taille = lignes*colonnes
    taille_null = 100*nb_null/taille
    taille_non_null = 100*(taille-nb_null)/taille
    print('\t-', nb_null, 'valeurs manquantes, ce qui représente', round(taille_null,2), '% du data set.')

    # Afficher la répartition du taux de valeurs manquantes
    nan_data = pd.DataFrame({'valeurs (%)': [''], 'non-null('+str(round(taille_non_null,2))+'%)': [taille_non_null], 'null('+str(round(taille_null,2))+'%)': [taille_null]})
    nan_data.set_index('valeurs (%)').plot(kind='barh', stacked=True, color=['green', 'orange'], figsize=(8,2), fontsize=12)
    plt.xlabel('%')

def afficher_echantillon(data, n=0.00002):
    """ Afficher un sous-échantillon aléatoire """
    print('Voici un sous-échantillon aléatoire :')
    return data.sample(frac=n, random_state=np.random.seed())

def valeurs_manquantes(data):
    """ Retourner les valeurs manquantes d'un data frame/set """
    return data[data.isnull().any(axis=1)]

## 1.3 Récupération du Data Set

In [ ]:
# Ouverture du fichier CSV en téléchargement : 
df = dd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t', assume_missing=True, low_memory=low_memory, 
dtype={'allergens': 'object',
       'allergens_fr': 'object',
       'categories': 'object',
       'categories_fr': 'object',
       'categories_tags': 'object',
       'cities': 'object',
       'cities_tags': 'object',
       'code': 'object',
       'created_t': 'object',
       'emb_codes': 'object',       
       'emb_codes_tags': 'object',
       'first_packaging_code_geo': 'object',
       'generic_name': 'object',
       'image_small_url': 'object',
       'image_url': 'object',
       'ingredients_from_palm_oil_tags': 'object',
       'ingredients_that_may_be_from_palm_oil_tags': 'object',
       'labels': 'object',
       'labels_fr': 'object',
       'labels_tags': 'object',
       'last_modified_t': 'object',
       'main_category': 'object',
       'main_category_fr': 'object',
       'manufacturing_places': 'object',
       'manufacturing_places_tags': 'object',
       'origins': 'object',
       'origins_tags': 'object',
       'packaging': 'object',
       'packaging_tags': 'object',
       'pnns_groups_1': 'object',
       'pnns_groups_2': 'object',
       'purchase_places': 'object',
       'stores': 'object',
       'traces': 'object',
       'traces_fr': 'object',
       'traces_tags': 'object',          
       'created_datetime': 'object',
       'last_modified_datetime': 'object',
       }) #le fichier doit être placé dans le même répertoire que ce notebook

if not low_memory:
       df = df.persist() #Si assez de RAM dispo, cele permettera d'accéler les futurs traitements

## Structure du data set

In [ ]:
df

In [ ]:
df.head(1)

In [ ]:
df.columns.tolist()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.shape[0].compute()

In [ ]:
df.describe().compute()  

In [ ]:
verifier_taille(df)

In [ ]:
afficher_echantillon(df).iloc[:,30:50].compute()

In [ ]:
valeurs_manquantes(df).compute()

## Transformation des colonnes dates

In [ ]:
dates_t = ['created_t', 'last_modified_t']
for date in dates_t:
    df[date] = dd.to_datetime(df[date], unit='s')

In [ ]:
df.dtypes

In [ ]:
df = df.drop(columns=['created_datetime', 'last_modified_datetime'])

In [ ]:
df.dtypes

In [ ]:
afficher_echantillon(df).compute()

In [ ]:
# mask = (df['created_t'].str.contains('\D')).replace(np.nan, False)
# df[mask].compute()

In [ ]:
# mask = (df['code'].str.contains('\D')).replace(np.nan, False)
# df[mask].compute()

In [ ]:
# df = df[df['code'].notnull()]

In [ ]:
# mask = (df['code'].isna()).replace(np.nan, False)
# df[mask].compute()